In [1]:
import pandas as pd
import random

pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_columns', 50)

In [2]:
# rolls
roll = {
    '📏': 'range',
    '❤': 'damage',
    '⚡': 'surge',
    '❌': 'miss',
}

red = ['❌', '❤❤❤❤', '❤❤📏📏', '❤📏📏⚡', '❤❤❤📏', '❤❤❤📏⚡']
blue = ['❌', '📏📏📏📏⚡', '❤📏📏📏⚡', '❤📏📏📏⚡', '❤❤📏📏', '❤❤📏']
white = ['❌', '❤📏📏📏⚡', '❤📏📏📏⚡', '❤❤❤📏⚡', '❤❤❤📏⚡', '❤❤📏📏']
yellow = ['📏📏⚡', '📏⚡', '📏📏⚡', '📏📏❤', '📏📏📏', '📏📏📏']
green = ['❤❤❤', '❤❤⚡', '❤❤❤', '❤❤⚡', '❤❤📏', '❤📏']
black = ['❤📏', '❤📏', '❤📏', '', '⚡', '⚡']
clear = ['❌', '❌', '', '', '', '']

In [3]:
# dice classes

class Die:
    def __init__(self, colour, possible_results):
        self.colour = colour
        self.POSSIBLE_RESULTS = possible_results
    
    def roll(self):
        return random.choice(self.POSSIBLE_RESULTS)


class Dice(Die):
    def __init__(self, n_red=0, n_blue=0, n_white=0, n_green=0, n_yellow=0, n_black=0, n_clear=0):
        self._dice = list()
        self.get_dice(n_red, n_blue, n_white, n_green, n_yellow, n_black, n_clear)
        self.current_roll = None
    
    def get_dice(self, n_red, n_blue, n_white, n_green, n_yellow, n_black, n_clear):
        for colour in ['red', 'blue', 'white', 'green', 'yellow', 'black', 'clear']:
            self._dice = self._dice + [Die(colour, eval(colour)) for _ in range(eval('n_'+colour))]

    def roll(self):
        result = []
        for die in self._dice:
            die_result = die.roll()
            result.append(die_result)
        self.current_roll = result
        return self.current_roll


In [4]:
# monster class

class Monster:
    """Monsters don't use effects (surges just lead to threat, which isn't tracked in this AI variant)"""
    def __init__(self, dice_obj, modifiers=None, type=None):
        self.dice = dice_obj
        self.modifiers = modifiers
        self.current_roll = None
        self.current_totals = None
        self.type = type
        self.roll()
        
    def roll(self):
        if self.dice:
            result = self.dice.roll()
        else:
            result = []
        if self.modifiers:
            modifier_list = []
            for k,v in self.modifiers.items():
                for _ in range(v):
                    modifier_list.append(k)
            result.extend(modifier_list)
        self.current_roll = result    
        self.calc_totals()
        return self.current_roll
    
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()
            
    def calc_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    final_results[result] = final_results.get(result, 0) + 1
        self.current_totals = final_results

    def print_totals(self):
        if self.current_totals == None:
            self.calc_totals()
        for result in self.current_totals.items():
            print(result)
        print()

    def results(self):        
        results = []
        for r in self.current_roll:
            results += [x for x in r]
        results = ''.join(sorted(results, reverse=True))

        columns = ['❌', '❤', '📏', '⚡']
        output = pd.DataFrame([[results.count('❌'), results.count('❤'), results.count('📏'), results.count('⚡')]], columns=columns, dtype='int64')

        if self.type == 'melee':
            output = output.drop('📏', axis=1)

        return output


In [5]:
# simulation functions

def simulation(obj, n_trials=100):
    obj.roll()
    results = pd.DataFrame(obj.results().max()).T
    for _ in range(n_trials-1):
        obj.roll()
        results = pd.concat([results, pd.DataFrame(obj.results().max()).T])
    return results

def summarise_simulation(dataframe, percentiles=[0.01]+[x/20 for x in range(1, 20)] + [0.99]):
    cols = ['mean', 'std']+[str(int(x*100))+'%' for x in percentiles]+['max']
    output = dataframe.describe(percentiles=percentiles).T
    return pd.concat([output.iloc[:, 1:3], output.iloc[:, 3:].astype(int)], axis=1)[cols]

def print_simulation_results(kwargs_dict, n_trials=1000):
    for monster, kwargs in kwargs_dict.items():
        print()
        print(monster)
        display(summarise_simulation(simulation(Monster(**kwargs), n_trials=n_trials)))

def calculate_range(kwargs_dict, n_trials=100, n_percentile=50):
    percentile = str(n_percentile)+'%'
    output_dict = dict()
    for monster, kwargs in kwargs_dict.items():
        if kwargs.get('type', False) and kwargs['type'] == 'melee':
            output_dict[monster] = 0
        else:
            df = summarise_simulation(simulation(Monster(**kwargs), n_trials=n_trials))
            output_dict[monster] = df.loc[['📏'], [percentile]].iloc[0].iloc[0]

    return output_dict

In [6]:
# monster kwargs

monsters = {
    'red_die': {
        'dice_obj': Dice(n_red=1),
    },
    'blue_die': {
        'dice_obj': Dice(n_blue=1),
    },
    'white_die': {
        'dice_obj': Dice(n_white=1),
    },
    'green_die': {
        'dice_obj': Dice(n_green=1),
    },
    'yellow_die': {
        'dice_obj': Dice(n_yellow=1),
    },
    'black_die': {
        'dice_obj': Dice(n_black=1),
    },
    'clear_die': {
        'dice_obj': Dice(n_clear=1),
    },
    'beastman': {
        'dice_obj': Dice(n_red=1, n_green=1),
        'modifiers': {'❤': 1},
        'type': 'melee'
    },
    'beastman_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=1),
        'modifiers': {'❤': 2},
        'type': 'melee'
    },
}

In [7]:
print_simulation_results(monsters, n_trials=100)


red_die


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.1,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
❤,2.2,1.3,0,0,0,0,1,1,1,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4
📏,1.0,0.8,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2
⚡,0.4,0.5,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1



blue_die


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
❤,1.0,0.8,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2
📏,2.1,1.3,0,0,0,0,1,1,1,2,2,2,3,3,3,3,3,3,3,3,3,4,4,4
⚡,0.5,0.5,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1



white_die


...

,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.4,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
❤,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
📏,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
⚡,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



beastman


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
❤,5.4,1.6,2,3,3,4,4,4,5,5,5,5,6,6,6,6,6,7,7,7,7,8,8,8
⚡,0.6,0.7,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2



beastman_master


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
❤,6.9,1.6,4,4,4,5,5,6,6,6,7,7,7,7,8,8,8,8,8,9,9,9,10,10
⚡,0.9,0.9,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,3,3,3


In [8]:
# range simulation - should we use 10%ile or 50%ile for range?
print('Range\n50th %ile: ', end='')
print(calculate_range(monsters))
print('10th %ile: ', end='')
print(calculate_range(monsters, n_percentile=10))

Range
50th %ile: {'red_die': 1, 'blue_die': 2, 'white_die': 2, 'green_die': 0, 'yellow_die': 2, 'black_die': 0, 'clear_die': 0, 'beastman': 0, 'beastman_master': 0}
10th %ile: {'red_die': 0, 'blue_die': 0, 'white_die': 0, 'green_die': 0, 'yellow_die': 1, 'black_die': 0, 'clear_die': 0, 'beastman': 0, 'beastman_master': 0}
